# Getting started with PySkinDose
Welcome! This is a getting-started guide for PySkinDose. It will be updated when more functionality is added or modified.

installationslänk

## PART I: Settings

We need to start our session with adding the class PyskindoseSettings for parsing of user defined settings:

In [ ]:
import json
from pyskindose import PyskindoseSettings, load_settings_example_json, print_available_human_phantoms
from pyskindose.main import main


once completed, we need to initialize all the user defined settings. Lets load a template of pre-loaded settings  (located in settings_example.json), then we can change each of the individual settings prior to calculating skin dose

In [ ]:
# Parse the settings to a setting class:
settings_json = load_settings_example_json()
settings = PyskindoseSettings(settings=settings_json)

Now, all the settings are populated in settings. We can print all user defined setting by typing `settings.print_parameters()`:

In [ ]:
settings.print_parameters()

As seen in the above output, `settings`  includes the sections __general__, __phantom__, and __plot__. 

You can access each of the settings in general by simply typing `settings.mode`, `settings.k_tab_val` etc. For the other sections, include the section name, e.g. `settings.phantom.patient_orientation`.


You can read up each by adressing the corresponding docstring with the `__doc__` attribute. E.g: `settings.__doc__` returns detailed descriptions of the setting in __general__, i.e., "mode", "k_tab_val",  "estimate_k_tab", and "rdsr_filename". 

Uncommany any of the following lines for more info on each of the subsections in settings.


In [ ]:
# UNCOMMENT ANY OF THE FOLLOWING LINES FOR DESCRIPTION ON THE VARIABLES IN SETTINGS

print(settings.__doc__) # uncomment this line to read up on settings.general
#print(settings.phantom.__doc__) # uncomment this line to read up on settings.phantom
#print(settings.phantom.patient_offset.__doc__)  # uncomment this line to read up on settings.phantom.patient_offset
#print(settings.phantom.dimension.__doc__) # uncomment this line to read up on settings.phantom.dimension
#print(settings.plot.__doc__) # uncomment this line to read up on settings.plot


## PART II: Plot setup

In [ ]:
#This section uses a lot of quite memory heavy interactive .html plots. 
#If you are low on available RAM memory, uncommen the following code block. 
#This will export all the plots to separate browser tabs so that you can close the each plot prior to creating a new one

#import plotly.io as pio
#pio.renderers.default = "browser"

Now, lets have a look on the geometry in which the skin dose is calculated. PySkinDose has a built in mode for plotting the setup (i.e. phantom, and its position upon the patient support table. This is accessed by running `main` with `settings.mode` = "plot_setup".

In [ ]:
settings.mode = 'plot_setup'
settings.phantom.model="cylinder"
main(settings=settings)

In the above output, you will see the phantom and its position upon the support table. The X-ray source, beam, and image receptor is also added in standard position (AP1=AP2=0)
There are a few things one should adress prior to conducting any skin dose estimation.

1. **Phantom selection**: Select you phantom by setting `settings.phantom.phantom_type` to either "plane", "cylinder", or "human". If you select "human", you need to specify what human phantom to use. The different phantoms can be found in source/phantom/phantom_data. Our current default phantom is "hudfrid.stl". Several more advanced phantoms (e.g. for neurointerventions) are in development. If you select any of the mathematical phantoms (plane or cylinder), then you can tweek settings such as length and radius in settings.phantom.dimension. You can also change the dimension of the patient support table and pad.
2. Patient 
3. Third item
4. Fourth item


In [ ]:
settings.mode = 'plot_setup'
settings.phantom.model="plane"
main(settings=settings)

Cylindrical phantoms are great for general purposes but for skin dose map visualisation based patient data a human phantom is the best option. Use `print_available_human_phantoms()` to show existing phantoms. The example below runs a `plot_setup` with a senior female phantom. Try using various phantoms by altering the `settings.phantom.human_mesh` input before moving on to the next section.

In [ ]:
print_available_human_phantoms()

In [ ]:
settings.mode = 'plot_setup'
settings.phantom.model="human"
settings.phantom.human_mesh = 'senior_female'
main(settings=settings)

## PART III: Plot procedure
A useful tool during analysis of individual procedures is to visualize the orientation and size of the x-ray beam in every irradiation event one by one. In the example below the `settings.mode` is changed to `plot_procedure`. Run the cell and use the slider in the plot to scroll through each irradiation event in the study. 

<b>NÅTT OM ATT DETTA KAN VARA MINNESKRÄVANDE?:</b>


In [ ]:
settings.mode = 'plot_procedure'
settings.phantom.model="cylinder"
main(settings=settings)

## PART IV: Calculate and plot a dosemap
This the original purpose of PySkinDose. To calculate and plot a dosemap on the phantom of choice use the `calculate_dose` mode and switch the `settings.plot.plot_dosemap` to `True`. A plot will illustrate the skin dose estimation map based on information in the RDSR-file together with available correction factors for the specific x-ray device. The plot is interactive so the phantom can rotate and if you hover the mouse over the skin an estimate of skin dose will appear.


In [ ]:
settings.mode = 'calculate_dose'
settings.plot.plot_dosemap = True
settings.phantom.model="cylinder"
main(settings=settings)

## PART V: Using a specific RDSR-file as input 
After trying the examples above it is time to use an RDSR-file of your own to estimate a skin dose map. Copy the RDSR file to this folder then add the filename in the `path` below. Note that you might have to move the phantom along the table to get the estimated skin dose map on the correct place of the patient. Uncomment the lines relating to `patient_offset` and add translations before running the cell again.

<b>HÄNVISNING TILL VAR RDSR-FILEN SKA LÄGGAS:</b>
 

In [ ]:
settings.mode = 'calculate_dose'
settings.plot.plot_dosemap = True
# path to sr = 

main(settings=settings, file_path=)

## Adding/Altering correction factors
The skin dose estimations above are correlated with large uncertainties. For a better estimation of skin doses one can add the specific `inherent_filtration` for the x-ray device and add a measured table and pad attenuation `k_tab_val` before altering the `estimate_k_tab` to `True`

In [ ]:
# TODO 
# init funktion för att printa tillgängliga exempel sr, printa även även absolut path (Max)
# gör klart rutan egen sr (max)
# textrutor (CG)
# db bug (JL)
# uppdaterar readme med work in progress på getting started, samt kontaktväg
# TODO funktion för print phantoms



# lägg till positioning parameters
# anpassa mätningar


In [ ]:
pio.renderers.default = "browser"
main(settings=settings) # högremain(settings=settings) # högre


In [ ]:
pio.renderers
